In [1]:
!pip install huggingface_hub transformers datasets zstandard sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 148.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 131.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import json
from huggingface_hub import hf_hub_download
import shutil
import os

# replace with your repo id
repo_id = "dilip025/llama-2-7b"
filename = "step_final_6262.pt"
filename_tokenizer = "tokenizerllma.model"
# filename_texts = "texts.json"   # 👈 your uploaded JSON file

# download checkpoint + tokenizer
cached_path = hf_hub_download(repo_id=repo_id, filename=filename)
cached_path_tokenizer = hf_hub_download(repo_id=repo_id, filename=filename_tokenizer)
# cached_path_texts = hf_hub_download(repo_id=repo_id, filename=filename_texts)  # 👈 download texts.json

# copy files locally
dest_path = f"./{filename}"
dest_path_tokenizer = f"./{filename_tokenizer}"
# dest_path_texts = f"./{filename_texts}"

shutil.copy(cached_path_tokenizer, dest_path_tokenizer)
shutil.copy(cached_path, dest_path)
# shutil.copy(cached_path_texts, dest_path_texts)

print("Copied tokenizer to:", dest_path_tokenizer)
print("Tokenizer exists?", os.path.exists(dest_path_tokenizer))

print("Copied checkpoint to:", dest_path)
print("Checkpoint exists?", os.path.exists(dest_path))

# print("Copied texts.json to:", dest_path_texts)
# print("Texts file exists?", os.path.exists(dest_path_texts))

# ✅ load the texts.json into Python variable
# with open(dest_path_texts, "r", encoding="utf-8") as f:
#     texts = json.load(f)

# print("Loaded texts:", len(texts))  # just print first 10 for sanity check


step_lightning_6100.pt:   0%|          | 0.00/3.27G [00:00<?, ?B/s]

tokenizerllma.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Copied tokenizer to: ./tokenizerllma.model
Tokenizer exists? True
Copied checkpoint to: ./step_lightning_6100.pt
Checkpoint exists? True


In [3]:
dest_path

'./step_lightning_4800.pt'

In [ ]:
from huggingface_hub import HfApi, HfFolder

# paste your HF token here
token = ""

# save token locally
HfFolder.save_token(token)

api = HfApi(token=token)  # initialize with token


# your repo id
repo_id = "dilip025/llama-2-7b"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import os
import json
from tqdm import tqdm
from typing import Dict, Any, List
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
from datasets import load_dataset
from transformers import LlamaTokenizer
from transformers import get_cosine_schedule_with_warmup
# from lion_pytorch import Lion

# --- 1. Satori "Akasha" Final Model Architecture ---

class RotaryPositionalEmbeddings(nn.Module):
    def __init__(self, dim, max_seq_len=8192, base=10000):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)
        self.max_seq_len = max_seq_len

    def forward(self, seq_len: int, device: torch.device):
        t = torch.arange(seq_len, device=device, dtype=self.inv_freq.dtype)
        freqs = torch.outer(t, self.inv_freq)
        return torch.cat((freqs, freqs), dim=-1).cos(), torch.cat((freqs, freqs), dim=-1).sin()

    def apply_rotary_emb(self, x, cos, sin):
        def rotate_half(x):
            x1, x2 = x[..., :x.shape[-1]//2], x[..., x.shape[-1]//2:]
            return torch.cat((-x2, x1), dim=-1)
        
        # FIX 1: Simplified and robust RoPE application
        def apply_rotary(tensor, cos, sin):
            # Adjust shape for broadcasting based on tensor dimensions
            if tensor.ndim == 4: # For Q/K in attention (B, H, T, D_head)
                cos = cos.unsqueeze(1)
                sin = sin.unsqueeze(1)
            # No unsqueeze needed for chunk summaries (B, NumChunks, D_model)
            return (tensor * cos) + (rotate_half(tensor) * sin)

        if isinstance(x, tuple):
            q, k = x
            return apply_rotary(q, cos, sin), apply_rotary(k, cos, sin)
        
        return apply_rotary(x, cos, sin)

class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))
    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
    def forward(self, x):
        return self._norm(x.float()).type_as(x) * self.weight

class SwiGLU(nn.Module):
    def __init__(self, in_features, out_features, bias=False):
        super().__init__()
        self.linear_gate = nn.Linear(in_features, out_features, bias=bias)
        self.linear_up = nn.Linear(in_features, out_features, bias=bias)
        self.act_fn = nn.SiLU()
    def forward(self, x):
        return self.act_fn(self.linear_gate(x)) * self.linear_up(x)

class VivekaMoE(nn.Module):
    def __init__(self, d_model, n_experts=8, top_k=2, ff_mult=2):
        super().__init__()
        self.n_experts = n_experts
        self.top_k = top_k
        self.router = nn.Linear(d_model, n_experts, bias=False)
        ff_dim = int(d_model * ff_mult * 4 / 3)
        self.experts = nn.ModuleList([
            nn.Sequential(
                SwiGLU(d_model, ff_dim),
                nn.Linear(ff_dim, d_model, bias=False)
            ) for _ in range(n_experts)
        ])
        self.aux_loss_coef = 0.01
    def forward(self, x):
        B, T, C = x.shape
        x_flat = x.view(-1, C)
        router_logits = self.router(x_flat)
        routing_weights, selected_experts = torch.topk(router_logits, self.top_k, dim=-1)
        routing_weights = F.softmax(routing_weights, dim=-1, dtype=torch.float32)
        p = F.softmax(router_logits, dim=-1, dtype=torch.float32)
        f = p.mean(dim=0)
        load_balancing_loss = self.aux_loss_coef * self.n_experts * torch.sum(f * f)
        final_output = torch.zeros_like(x_flat)
        for i in range(self.n_experts):
            token_mask = (selected_experts == i).any(dim=-1)
            if token_mask.any():
                expert_input = x_flat[token_mask]
                expert_output = self.experts[i](expert_input)
                routing_subset = routing_weights[token_mask]
                weight_mask = (selected_experts[token_mask] == i)
                final_output[token_mask] += (expert_output * routing_subset[weight_mask].unsqueeze(-1))
        return final_output.view(B, T, C), load_balancing_loss

# First, create a dedicated SwiGLU FeedForward module for clarity.
# This is the same structure used inside your MoE experts.
class SwiGLUFeedForward(nn.Module):
    def __init__(self, d_model, ff_mult=2):
        super().__init__()
        # The hidden dim is often a multiple of d_model, adjusted for SwiGLU best practices.
        ff_dim = int(d_model * ff_mult * 2 / 3)
        self.w1 = nn.Linear(d_model, ff_dim, bias=False)
        self.w3 = nn.Linear(d_model, ff_dim, bias=False)
        self.w2 = nn.Linear(ff_dim, d_model, bias=False)

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

class HierarchicalGatedPooling(nn.Module):
    def __init__(self, d_model, n_heads, dropout, chunk_size=256, num_summary_vectors=4):
        super().__init__()
        # ... (self.chunk_size, n_heads, d_head, num_summary_vectors are the same) ...
        self.chunk_size = chunk_size
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.num_summary_vectors = num_summary_vectors

        # --- MODIFICATION: Build a custom summarizer block ---
        self.summary_token = nn.Parameter(torch.randn(1, 1, d_model))
        
        # 1. Self-Attention Layer
        self.summarizer_attn = nn.MultiheadAttention(
            embed_dim=d_model, 
            num_heads=n_heads, 
            dropout=dropout, 
            batch_first=True
        )
        self.summarizer_norm1 = RMSNorm(d_model)

        # 2. SwiGLU Feed-Forward Layer
        self.summarizer_ffn = SwiGLUFeedForward(d_model)
        self.summarizer_norm2 = RMSNorm(d_model)
        # --- END OF MODIFICATION ---
        
        # ... (the rest of the __init__ is the same) ...
        self.query_proj = nn.Linear(d_model, d_model * num_summary_vectors, bias=False)
        self.summary_proj = nn.Linear(d_model * num_summary_vectors, d_model, bias=False)
        self.to_kv = nn.Linear(d_model, 2 * d_model, bias=False)
        self.to_g = nn.Linear(d_model, d_model, bias=True)
        self.out_proj = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.norm = RMSNorm(d_model)
        
    def forward(self, x, rope_cos, rope_sin, rope_applier):
        # ... (padding and chunking logic is the same) ...
        B, T, C = x.shape
        padding_needed = (self.chunk_size - T % self.chunk_size) % self.chunk_size
        if padding_needed > 0:
            x = F.pad(x, (0, 0, 0, padding_needed))
        
        chunks = x.view(B, -1, self.chunk_size, C)
        B_chunks, num_chunks, chunk_len, C = chunks.shape
        
        # --- MODIFICATION: Use the custom summarizer block ---
        summary_token = self.summary_token.expand(B_chunks * num_chunks, -1, -1)
        chunks_flat = chunks.view(B_chunks * num_chunks, chunk_len, C)
        chunk_with_token = torch.cat([summary_token, chunks_flat], dim=1)

        # Custom Transformer Block Logic
        # 1. Pre-Norm and Self-Attention
        normed_chunk = self.summarizer_norm1(chunk_with_token)
        attn_output, _ = self.summarizer_attn(normed_chunk, normed_chunk, normed_chunk, need_weights=False)
        # Residual Connection
        post_attn = chunk_with_token + attn_output

        # 2. Pre-Norm and SwiGLU FFN
        normed_post_attn = self.summarizer_norm2(post_attn)
        ffn_output = self.summarizer_ffn(normed_post_attn)
        # Residual Connection
        summarizer_output = post_attn + ffn_output

        # The output corresponding to the first token is our learned summary
        learned_summary = summarizer_output[:, 0, :] 
        # --- END OF MODIFICATION ---

        # ... (The rest of the forward pass is exactly the same) ...
        chunk_content_rep = learned_summary.view(B_chunks, num_chunks, C)
        queries_unshaped = self.query_proj(chunk_content_rep)
        q = queries_unshaped.view(B * num_chunks, self.num_summary_vectors, self.n_heads, self.d_head).transpose(1, 2)
        k, v = self.to_kv(chunks).chunk(2, dim=-1)
        k = k.view(B * num_chunks, self.chunk_size, self.n_heads, self.d_head).transpose(1, 2)
        v = v.view(B * num_chunks, self.chunk_size, self.n_heads, self.d_head).transpose(1, 2)
        multi_vector_summaries = F.scaled_dot_product_attention(q, k, v)
        summaries_flat = multi_vector_summaries.transpose(1, 2).reshape(B, num_chunks, C * self.num_summary_vectors)
        chunk_summaries = self.summary_proj(summaries_flat)
        chunk_summaries_with_rope = rope_applier(chunk_summaries, rope_cos, rope_sin)
        gate_chunks = torch.sigmoid(self.to_g(chunk_summaries_with_rope))
        gated_v_chunks = chunk_summaries_with_rope * gate_chunks
        cumulative_sum = torch.cumsum(gated_v_chunks, dim=1)
        cumulative_gates = torch.cumsum(gate_chunks, dim=1)
        chunk_context = cumulative_sum / (cumulative_gates + 1e-9)
        zero_pad = torch.zeros(B, 1, C, device=x.device, dtype=x.dtype)
        padded_chunk_context = torch.cat([zero_pad, chunk_context], dim=1)
        prev_chunk_context = padded_chunk_context[:, :-1, :]
        chunk_context_expanded = prev_chunk_context.unsqueeze(2).expand(-1, -1, self.chunk_size, -1)
        token_context = chunk_context_expanded.reshape(B, -1, C)
        if padding_needed > 0:
            token_context = token_context[:, :T, :]
        normed_context = self.norm(token_context)
        output = self.out_proj(normed_context)
        return self.dropout(output)

class LocalSlidingWindowAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout, window_size=1024):
        super().__init__()
        self.n_heads = n_heads
        self.window_size = window_size
        self.qkv_proj = nn.Linear(d_model, 3 * d_model, bias=False)
        self.out_proj = nn.Linear(d_model, d_model, bias=False)
        self.dropout_p = dropout
    def forward(self, x, rope_cos, rope_sin, rope_applier):
        B, T, C = x.shape
        q, k, v = self.qkv_proj(x).chunk(3, dim=-1)
        
        # FIX 3: Correctly unpack the tensors instead of creating a generator.
        q = q.view(B, T, self.n_heads, C // self.n_heads)
        k = k.view(B, T, self.n_heads, C // self.n_heads)
        v = v.view(B, T, self.n_heads, C // self.n_heads)
        
        q, k = rope_applier((q, k), rope_cos, rope_sin)
        
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
        
        mask = torch.full((T, T), float("-inf"), device=x.device, dtype=q.dtype)
        rows = torch.arange(T, device=x.device)[:, None]
        cols = torch.arange(T, device=x.device)[None, :]
        window_mask = (rows >= cols) & (rows - cols < self.window_size)
        mask.masked_fill_(window_mask, 0.0)

        attn_output = F.scaled_dot_product_attention(
            q, k, v, attn_mask=mask, dropout_p=self.dropout_p if self.training else 0.0)
        
        output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        return self.out_proj(output)

class AkashaBlock(nn.Module):
    def __init__(self, config: Dict[str, Any]):
        super().__init__()
        self.norm = RMSNorm(config['d_model'])
        self.purusha_path = HierarchicalGatedPooling(config['d_model'], config['n_heads'], config['dropout'], config['chunk_size'],config['num_summary_vectors'])
        self.prakriti_path = LocalSlidingWindowAttention(config['d_model'], config['n_heads'], config['dropout'], config['window_size'])
        self.fusion_gate = nn.Linear(config['d_model'], 2 * config['d_model'], bias=False)
        self.moe = VivekaMoE(config['d_model'], config['n_experts'], config['top_k'])
        self.moe_norm = RMSNorm(config['d_model'])
        self.residual_dropout = nn.Dropout(config['dropout'])
    def forward(self, x, token_rope, chunk_rope):
        normed_x = self.norm(x)
        purusha_out = self.purusha_path(normed_x, chunk_rope[0], chunk_rope[1], chunk_rope[2])
        prakriti_out = self.prakriti_path(normed_x, token_rope[0], token_rope[1], token_rope[2])
        gate1, gate2 = self.fusion_gate(normed_x).chunk(2, dim=-1)
        gate1, gate2 = torch.sigmoid(gate1), torch.sigmoid(gate2)
        fused_out = gate1 * purusha_out + gate2 * prakriti_out
        x = x + self.residual_dropout(fused_out)
        moe_out, moe_loss = self.moe(self.moe_norm(x))
        x = x + self.residual_dropout(moe_out)
        return x, moe_loss

class Satori_Akasha_Model(nn.Module):
    def __init__(self, config: Dict[str, Any]):
        super().__init__()
        self.config = config
        self.token_embedding = nn.Embedding(config['vocab_size'], config['d_model'])
        self.embedding_dropout = nn.Dropout(config['dropout'])
        self.token_rope = RotaryPositionalEmbeddings(config['d_model'] // config['n_heads'], config['block_size'])
        self.chunk_rope = RotaryPositionalEmbeddings(config['d_model'], config['max_chunks'])
        self.layers = nn.ModuleList([AkashaBlock(config) for _ in range(config['n_layers'])])
        self.final_norm = RMSNorm(config['d_model'])
        self.output_layer = nn.Linear(config['d_model'], config['vocab_size'], bias=False)
        self.output_layer.weight = self.token_embedding.weight
        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_normal_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.xavier_normal_(module.weight)
    def forward(self, input_ids, labels=None):
        x = self.embedding_dropout(self.token_embedding(input_ids))
        token_rope_cos, token_rope_sin = self.token_rope(x.shape[1], x.device)
        num_chunks = (x.shape[1] + self.config['chunk_size'] - 1) // self.config['chunk_size']
        chunk_rope_cos, chunk_rope_sin = self.chunk_rope(num_chunks, x.device)
        token_rope_params = (token_rope_cos, token_rope_sin, self.token_rope.apply_rotary_emb)
        chunk_rope_params = (chunk_rope_cos, chunk_rope_sin, self.chunk_rope.apply_rotary_emb)
        total_moe_loss = torch.tensor(0.0, device=x.device)
        for layer in self.layers:
            x, moe_loss = layer(x, token_rope_params, chunk_rope_params)
            total_moe_loss += moe_loss
        logits = self.output_layer(self.final_norm(x))
        loss_dict = {}
        if labels is not None:
            task_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))
            loss_dict = {"task_loss": task_loss, "moe_aux_loss": total_moe_loss / len(self.layers)}
        return logits, loss_dict


# --- 2. Training Configuration ---
def get_satori_akasha_config():
    block_size = 2048
    chunk_size = 256
    config = {
        "model_name": "Satori-Akasha-250M",
        "d_model": 768, "n_layers": 10, "n_heads": 8,
        "window_size": 256, "chunk_size": chunk_size,
        "max_chunks": 8192,
        "n_experts": 2, "top_k": 2, "dropout": 0.05,
        "vocab_size": 32000, "block_size": block_size,
        "num_summary_vectors": 4,
        "tokenizer_path": dest_path_tokenizer,
        "num_epochs": 1, "batch_size": 22, "grad_accum_steps": 35,
        "precision": "bf16", "max_grad_norm": 1.0,
        "optimizer": "AdamW", "learning_rate": 3e-4, "weight_decay": 0.02,
        "beta1": 0.9, "beta2": 0.95,
        "num_train_samples": 460000, "skip_train_samples": 130000+130000+120000+120000+120000+130000+125000+125000+130000+130000+130000+330000+430000,
        "lr_scheduler": "cosine_with_warmup", "warmup_steps": 100,
        "output_dir": "satori_akasha_checkpoints",
        "log_interval_steps": 10, "save_interval_steps": 600,
        "resume_training": False,
        "resume_from_checkpoint": True, 
        
        # 💡 This is our GLOBAL training plan. Never change this number mid-training.
        # "total_training_steps": 45200,
        "total_training_steps": 11000,
    }
    return config

# --- 3. Production-Ready Training Loop ---

# --- 6. Data Pipeline ---
def load_training_data(val_size=10000, skip_size=0):
    streamed_dataset = load_dataset("HuggingFaceFW/fineweb", split="train", streaming=True).skip(skip_size)
    # streamed_dataset = load_dataset("allenai/c4","en", split="train", streaming=True, trust_remote_code=True).skip(skip_size)
    texts = []
    print(f"Collecting {val_size:,} samples from pile (skipping first {skip_size:,})...")
    
    for ex in tqdm(streamed_dataset, total=val_size, desc="Downloading pajama"):
        texts.append(ex["text"])
        if len(texts) >= val_size:
            break

    print(f"📦 Loaded {len(texts):,} samples.")
    return texts

class TextDatasetForLM(Dataset):
    def __init__(self, texts: List[str], tokenizer, block_size: int, batch_size: int = 4048):
        all_token_ids = []
        self.eos_token_id = tokenizer.eos_token_id

        print(f"Tokenizing {len(texts):,} texts in batches of {batch_size}...")
        for i in tqdm(range(0, len(texts), batch_size), desc="Batch Tokenizing"):
            batch_texts = texts[i:i + batch_size]
            encodings = tokenizer(
                batch_texts,
                truncation=False,
                padding=False,
                add_special_tokens=False,
                return_attention_mask=False,
            )  # HF returns dict

            for input_ids in encodings["input_ids"]:
                if input_ids:
                    all_token_ids.extend(input_ids + [self.eos_token_id])  # Append EOS

        # Convert long stream of tokens into block-sized chunks
        self.examples = []
        for i in range(0, len(all_token_ids) - block_size + 1, block_size):
            self.examples.append(torch.tensor(all_token_ids[i:i + block_size], dtype=torch.long))

        print(f"✅ Prepared {len(self.examples):,} samples of size {block_size}.")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        x = self.examples[i]
        return x[:-1], x[1:]  # Input and target


def collate_batch(batch):
    inputs, labels = zip(*batch)
    return torch.stack(inputs), torch.stack(labels)


# No changes needed here, just for confirmation
def setup(config):
    """Sets up device, tokenizer, and directories."""
    config['device'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if config['precision'] == 'bf16' and not torch.cuda.is_bf16_supported():
        print("Warning: bfloat16 not supported on this device, falling back to fp32.")
        config['precision'] = 'fp32'

    try:
        tokenizer = LlamaTokenizer.from_pretrained(config['tokenizer_path'])
        config['vocab_size'] = tokenizer.vocab_size
        config['pad_token_id'] = tokenizer.pad_token_id

        print("vocab size", tokenizer.vocab_size)

        if config['pad_token_id'] is None:
            print("Info: No <pad> token defined, setting pad_token_id = eos_token_id.")
            config['pad_token_id'] = tokenizer.eos_token_id
    except Exception as e:
        print(f"Could not load tokenizer from {config['tokenizer_path']}. Error: {e}")
        tokenizer = None

    os.makedirs(config['output_dir'], exist_ok=True)
    
    return config, tokenizer

def create_model(config):
    """Creates and compiles the Satori-v9 model."""
    model = Satori_Akasha_Model(config)
    
    # Best Practice: Use torch.compile for significant speedup
    if torch.__version__ >= "2.0.0":
        print("Compiling the model with torch.compile()...")
        model = torch.compile(model, backend="aot_eager")
        
    return model.to(config['device'])

def create_optimizer_and_scheduler(model, config, num_training_steps):
    """
    Creates an AdamW optimizer and a standard cosine learning rate scheduler 
    from the Hugging Face transformers library.
    """
    # Don't apply weight decay to bias or LayerNorm/RMSNorm parameters
    param_dict = {pn: p for pn, p in model.named_parameters() if p.requires_grad}
    decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
    nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
    
    optim_groups = [
        {'params': decay_params, 'weight_decay': config['weight_decay']},
        {'params': nodecay_params, 'weight_decay': 0.0}
    ]
    
    # --- MODIFICATION: Use AdamW instead of Lion ---
    optimizer = torch.optim.AdamW(
        optim_groups,
        lr=config['learning_rate'],
        betas=(config.get('beta1', 0.9), config.get('beta2', 0.95)) # Your betas are good
    )
    # --- END OF MODIFICATION ---
    
    # The scheduler remains the same
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=config['warmup_steps'],
        num_training_steps=num_training_steps
    )
    
    return optimizer, scheduler

def train(model, config, train_loader, optimizer, scheduler, resume_step):
    """The main training loop."""
    device = config['device']
    use_amp = config['precision'] in ['fp16', 'bf16']
    dtype = torch.bfloat16 if config['precision'] == 'bf16' else torch.float16
    scaler = GradScaler(enabled=(config['precision'] == 'fp16'))
    
    global_step = resume_step
    model.train()
    avg_loss = 0
    for epoch in range(config['num_epochs']):
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']}")
        
        for i, (input_ids, labels) in enumerate(pbar):
            input_ids, labels = input_ids.to(device), labels.to(device)
            with torch.autocast(device_type=device.type, dtype=dtype, enabled=use_amp):
                _, loss_dict = model(input_ids, labels)
                task_loss = loss_dict['task_loss']
                moe_loss = loss_dict['moe_aux_loss']
                total_loss = task_loss + moe_loss
                avg_loss +=total_loss.item()
                # Scale loss for gradient accumulation
                scaled_loss = total_loss / config['grad_accum_steps']

            # Backward pass
            scaler.scale(scaled_loss).backward()
            
            # Gradient accumulation step
            if (i + 1) % config['grad_accum_steps'] == 0:
                # Best Practice: Unscale gradients before clipping
                scaler.unscale_(optimizer)
                
                # Best Practice: Clip gradients to prevent explosion
                torch.nn.utils.clip_grad_norm_(model.parameters(), config['max_grad_norm'])
                
                # Optimizer and scheduler step
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad(set_to_none=True)
                
                global_step += 1
                
                # Logging
                if global_step % config['log_interval_steps'] == 0:
                    pbar.set_postfix({
                        "loss": f"{total_loss.item():.4f}",
                        "avg_loss":f"{avg_loss/(i+1)}",
                        "task": f"{task_loss.item():.4f}",
                        "moe": f"{moe_loss.item():.4f}",
                        "lr": f"{scheduler.get_last_lr()[0]:.6e}"
                    })
                    
                print({
                        "loss": f"{total_loss.item():.4f}",
                        "task": f"{task_loss.item():.4f}",
                        "avg_loss":f"{avg_loss/(i+1)}",
                        "moe": f"{moe_loss.item():.4f}",
                        "lr": f"{scheduler.get_last_lr()[0]:.6e}"
                    })

                # Checkpointing
                if global_step > 0 and global_step % config['save_interval_steps'] == 0:
                    checkpoint_path = os.path.join(config['output_dir'], f"step_{global_step}.pt")
                    torch.save({
                        "step": global_step,
                        "model_state_dict": model.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "scheduler_state_dict": scheduler.state_dict(),
                        "scaler_state_dict": scaler.state_dict(),
                        "config": config
                    }, checkpoint_path)
                    print(f"\n✅ Saved checkpoint to {checkpoint_path}")
                    # api.upload_folder(
                    # folder_path="satori_akasha_checkpoints",
                    # repo_id=repo_id,
                    # repo_type="model"
                    # )
    print("Successfully uploaded checkpoints to Hugging Face Hub!")

    checkpoint_path = os.path.join(config['output_dir'], f"step_final_{global_step}.pt")
    torch.save({
                        "step": global_step,
                        "model_state_dict": model.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "scheduler_state_dict": scheduler.state_dict(),
                        "scaler_state_dict": scaler.state_dict(),
                        "config": config
                    }, checkpoint_path)
    # api.upload_folder(
    # folder_path="satori_akasha_checkpoints",
    # repo_id=repo_id,
    # repo_type="model"
    # )
    print("Successfully uploaded checkpoints to Hugging Face Hub!")
    print(f"\n✅ Saved checkpoint to {checkpoint_path}")
            

    print("🎉 Training finished.")


if __name__ == '__main__': 
    # --- 1. Get Configuration ---
    config = get_satori_akasha_config()
    
    # --- 2. Setup Environment ---
    config, tokenizer = setup(config)
    print("--- Starting Training Run for Satori-v9 'Prakriti-Purusha' ---")
    print(json.dumps({k: str(v) if isinstance(v, torch.device) else v 
                  for k, v in config.items()}, indent=2))


    # # --- 3. Load Data ---
    # train_texts = load_training_data(val_size=config['num_train_samples'], skip_size=config['skip_train_samples'])
    # train_texts = texts
    # print(f"✅ Remaining texts after filtering: {len(train_texts):,}")
   
    
    

/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1938: FutureWarning: Calling LlamaTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


vocab size 32000
Info: No <pad> token defined, setting pad_token_id = eos_token_id.
--- Starting Training Run for Satori-v9 'Prakriti-Purusha' ---
{
  "model_name": "Satori-Akasha-250M",
  "d_model": 768,
  "n_layers": 10,
  "n_heads": 8,
  "window_size": 256,
  "chunk_size": 256,
  "max_chunks": 8192,
  "n_experts": 2,
  "top_k": 2,
  "dropout": 0.05,
  "vocab_size": 32000,
  "block_size": 2048,
  "num_summary_vectors": 4,
  "tokenizer_path": "./tokenizerllma.model",
  "num_epochs": 1,
  "batch_size": 22,
  "grad_accum_steps": 35,
  "precision": "bf16",
  "max_grad_norm": 1.0,
  "optimizer": "AdamW",
  "learning_rate": 0.0003,
  "weight_decay": 0.02,
  "beta1": 0.9,
  "beta2": 0.95,
  "num_train_samples": 460000,
  "skip_train_samples": 2150000,
  "lr_scheduler": "cosine_with_warmup",
  "warmup_steps": 100,
  "output_dir": "satori_akasha_checkpoints",
  "log_interval_steps": 10,
  "save_interval_steps": 600,
  "resume_training": false,
  "resume_from_checkpoint": "/kaggle/input/enhanc

In [5]:
from torch.utils.data import Subset, DataLoader

# Download file from Hub
local_path = hf_hub_download(repo_id=repo_id, filename="tokenized_train_datasetii.pt", repo_type="model")

# Load directly
train_dataset = torch.load(local_path, weights_only=False)

# DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    collate_fn=collate_batch,
    num_workers=4,
    pin_memory=True
)

tokenized_train_datasetii.pt:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

In [8]:
# --- 4. Create Model, Optimizer, and Scheduler ---
model = create_model(config)
checkpoint_path = dest_path # Assuming dest_path is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoints = torch.load(checkpoint_path, map_location=device)

# --- This part is good, create a fresh optimizer and scheduler ---
# It correctly uses the TOTAL training steps for the LR curve
config['warmup_steps'] = int(config['total_training_steps'] * 0.03)
optimizer, scheduler = create_optimizer_and_scheduler(model, config, config['total_training_steps'])

steps_already_done = 0
if config['resume_from_checkpoint']:
    print(f"🔄 Resuming training from checkpoint: {config['resume_from_checkpoint']}")
    
    # 1. Load model weights (Correct)
    model.load_state_dict(checkpoints['model_state_dict'])
    
    # 2. DO NOT load optimizer state to reset momentum (Correct)
    # optimizer.load_state_dict(checkpoints['optimizer_state_dict'])
    
    # 3. Get the step count from the checkpoint (Correct)
    steps_already_done = checkpoints.get('step', 0)
    
    # 4. Manually fast-forward the NEW scheduler to the correct step.
    #    DO NOT load its state_dict.
    print(f"⏩ Fast-forwarding a fresh scheduler by {steps_already_done} steps...")
    for _ in range(steps_already_done):
        scheduler.step()
        
    print(f"Scheduler is now at step {steps_already_done}. LR before kick: {scheduler.get_last_lr()[0]:.6e}")

    # 5. NOW, apply the LR kick to the optimizer.
    #    The scheduler will now correctly proceed from this new, higher LR.
    print("🚀 Applying learning rate kick...")
    for param_group in optimizer.param_groups:
        param_group['lr'] = 1.5e-4
        
    print(f"✅ Resumed successfully. Kicked LR is now: {optimizer.param_groups[0]['lr']:.6e}")

print(f"Total Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.2f}M")

# --- 5. Start Training ---
# Pass the correct resume step to the training loop
train(model, config, train_loader, optimizer, scheduler, resume_step=steps_already_done)

Compiling the model with torch.compile()...


/tmp/ipykernel_243/3606871262.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(config['precision'] == 'fp16'))


🔄 Resuming training from checkpoint: /kaggle/input/enhanced-purusha-prakirti/satori_akasha_checkpoints/step_final_2692.pt
⏩ Fast-forwarding a fresh scheduler by 6100 steps...
Scheduler is now at step 6100. LR before kick: 1.045725e-04
🚀 Applying learning rate kick...
✅ Resumed successfully. Kicked LR is now: 1.500000e-04
Total Parameters: 272.40M


Epoch 1/1:   0%|                                                            | 8/5703 [00:36<5:29:14,  3.47s/it]W0912 15:33:44.179000 243 site-packages/torch/_dynamo/convert_frame.py:1016] [2/8] torch._dynamo hit config.recompile_limit (8)
W0912 15:33:44.179000 243 site-packages/torch/_dynamo/convert_frame.py:1016] [2/8]    function: 'forward' (/tmp/ipykernel_243/2156210077.py:255)
W0912 15:33:44.179000 243 site-packages/torch/_dynamo/convert_frame.py:1016] [2/8]    last reason: 2/0: Cache line invalidated because L['token_rope'][2] got deallocated
W0912 15:33:44.179000 243 site-packages/torch/_dynamo/convert_frame.py:1016] [2/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0912 15:33:44.179000 243 site-packages/torch/_dynamo/convert_frame.py:1016] [2/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.
Epoch 1/1:   1%|▎                                                          | 35/5703 [00:53<1:01:56,  1.53it/s]

{'loss': '3.1710', 'task': '3.1610', 'avg_loss': '3.093504238128662', 'moe': '0.0100', 'lr': '1.045262e-04'}


...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ..._checkpoints/step_lightning_6250.pt:   3%|2         | 92.3MB / 3.27GB            

  ...asha_checkpoints/step_final_6262.pt:   0%|          |  552kB / 3.27GB            

Successfully uploaded checkpoints to Hugging Face Hub!

✅ Saved checkpoint to satori_akasha_checkpoints/step_final_6262.pt
🎉 Training finished.


In [9]:
api.upload_folder(
    folder_path="satori_akasha_checkpoints",
    repo_id=repo_id,
    repo_type="model"
)
print("Successfully uploaded checkpoints to Hugging Face Hub!")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...asha_checkpoints/step_final_6262.pt:   3%|3         |  101MB / 3.27GB            

  ..._checkpoints/step_lightning_6250.pt:   3%|3         |  101MB / 3.27GB            

No files have been modified since last commit. Skipping to prevent empty commit.


Successfully uploaded checkpoints to Hugging Face Hub!


In [ ]:
# def train(model, config, train_loader, optimizer, scheduler, resume_step):
#     """The main training loop."""
#     device = config['device']
#     use_amp = config['precision'] in ['fp16', 'bf16']
#     dtype = torch.bfloat16 if config['precision'] == 'bf16' else torch.float16
#     scaler = GradScaler(enabled=(config['precision'] == 'fp16'))
    
#     global_step = resume_step
#     model.train()
#     avg_loss = 0
#     for epoch in range(config['num_epochs']):
#         pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']}")
        
#         for i, (input_ids, labels) in enumerate(pbar):
#             input_ids, labels = input_ids.to(device), labels.to(device)
#             with torch.autocast(device_type=device.type, dtype=dtype, enabled=use_amp):
#                 _, loss_dict = model(input_ids, labels)
#                 task_loss = loss_dict['task_loss']
#                 moe_loss = loss_dict['moe_aux_loss']
#                 total_loss = task_loss + moe_loss
#                 avg_loss +=total_loss.item()
#                 # Scale loss for gradient accumulation
#                 scaled_loss = total_loss / config['grad_accum_steps']

#             # Backward pass
#             scaler.scale(scaled_loss).backward()
            
#             # Gradient accumulation step
#             if (i + 1) % config['grad_accum_steps'] == 0:
#                 # Best Practice: Unscale gradients before clipping
#                 scaler.unscale_(optimizer)
                
#                 # Best Practice: Clip gradients to prevent explosion
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), config['max_grad_norm'])
                
#                 # Optimizer and scheduler step
#                 scaler.step(optimizer)
#                 scaler.update()
#                 scheduler.step()
#                 optimizer.zero_grad(set_to_none=True)
                
#                 global_step += 1
                
#                 # Logging
#                 if global_step % config['log_interval_steps'] == 0:
#                     pbar.set_postfix({
#                         "loss": f"{total_loss.item():.4f}",
#                         "avg_loss":f"{avg_loss/(i+1)}",
#                         "task": f"{task_loss.item():.4f}",
#                         "moe": f"{moe_loss.item():.4f}",
#                         "lr": f"{scheduler.get_last_lr()[0]:.6e}"
#                     })
                    
#                 print({
#                         "loss": f"{total_loss.item():.4f}",
#                         "task": f"{task_loss.item():.4f}",
#                         "avg_loss":f"{avg_loss/(i+1)}",
#                         "moe": f"{moe_loss.item():.4f}",
#                         "lr": f"{scheduler.get_last_lr()[0]:.6e}"
#                     })

#                 # Checkpointing
#                 if global_step > 0 and global_step % 50 == 0:
#                     checkpoint_path = os.path.join(config['output_dir'], f"step_lightning_{global_step}.pt")
#                     torch.save({
#                         "step": global_step,
#                         "model_state_dict": model.state_dict(),
#                         "optimizer_state_dict": optimizer.state_dict(),
#                         "scheduler_state_dict": scheduler.state_dict(),
#                         "scaler_state_dict": scaler.state_dict(),
#                         "config": config
#                     }, checkpoint_path)
#                     print(f"\n✅ Saved checkpoint to {checkpoint_path}")
#                     api.upload_folder(
#                     folder_path="satori_akasha_checkpoints",
#                     repo_id=repo_id,
#                     repo_type="model"
#                     )
#     print("Successfully uploaded checkpoints to Hugging Face Hub!")

#     checkpoint_path = os.path.join(config['output_dir'], f"step_final_{global_step}.pt")
#     torch.save({
#                         "step": global_step,
#                         "model_state_dict": model.state_dict(),
#                         "optimizer_state_dict": optimizer.state_dict(),
#                         "scheduler_state_dict": scheduler.state_dict(),
#                         "scaler_state_dict": scaler.state_dict(),
#                         "config": config
#                     }, checkpoint_path)
#     api.upload_folder(
#     folder_path="satori_akasha_checkpoints",
#     repo_id=repo_id,
#     repo_type="model"
#     )
#     print("Successfully uploaded checkpoints to Hugging Face Hub!")
#     print(f"\n✅ Saved checkpoint to {checkpoint_path}")
            

#     print("🎉 Training finished.")

In [9]:
dest_path

'./step_final_4769.pt'